In [7]:

''' Denne Python-koden genererer en PDF som inneholder en serie sekvensielt nummererte konsertbilletter.
Hver billett inkluderer en unik billett-ID, QR-koder og et bilde levert av en bruke '''

import qrcode
from PIL import Image, ImageDraw, ImageFont
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
import os

def generate_ticket_id(prefix, start_number, total_tickets):
    tickets = []
    for i in range(start_number, start_number + total_tickets):
        ticket_id = f"{prefix}-{i:05d}"  # Ticket ID with padding
        tickets.append(ticket_id)
    return tickets

def generate_qr_code(ticket_id, qr_size=(100, 100)):
    qr = qrcode.QRCode(
        version=1,  # Sise of the QR code
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(ticket_id)  # Add the ticket ID to the QR code
    qr.make(fit=True)
    
    img = qr.make_image(fill='black', back_color='white')
    img = img.resize(qr_size, Image.LANCZOS)
    return img

def create_ticket_image(ticket_id, qr_code_left_img, qr_code_right_img, provided_image):
    provided_img = Image.open(provided_image)

    # Set ticket dimensions based on the provided image
    img_width, img_height = provided_img.size
    ticket_width = img_width + 50
    ticket_height = img_height + 50
    padding = 20

    # Position the QR codes
    qr_code_left_position = (10, img_height - qr_code_left_img.height - 10)
    qr_code_right_position = (img_width - qr_code_right_img.width - 10, img_height - qr_code_right_img.height - 10)

    # Add QR codes to the provided image
    provided_img.paste(qr_code_left_img, qr_code_left_position)
    provided_img.paste(qr_code_right_img, qr_code_right_position)

    # Create a new ticket image with padding
    ticket_image = Image.new('RGB', (ticket_width, ticket_height), color='white')
    draw = ImageDraw.Draw(ticket_image)
    font = ImageFont.load_default()

    # Add ticket information
    draw.text((padding, padding), f"Ticket ID: {ticket_id}", font=font, fill='black')

    # Paste the final ticket layout onto the blank canvas
    ticket_image.paste(provided_img, (padding, padding))
    return ticket_image

def generate_tickets_pdf(prefix, start_number, total_tickets, output_pdf, provided_image):
    tickets = generate_ticket_id(prefix, start_number, total_tickets)
    
    # Create a PDF canvas
    c = canvas.Canvas(output_pdf, pagesize=A4)
    pdf_width, pdf_height = A4

    for ticket_id in tickets:
        # Generate QR codes
        qr_code_left_img = generate_qr_code(ticket_id, qr_size=(130, 130))
        qr_code_right_img = generate_qr_code(ticket_id, qr_size=(130, 130))

        # Generate ticket image with QR codes
        ticket_image = create_ticket_image(ticket_id, qr_code_left_img, qr_code_right_img, provided_image)

        # Resize ticket image to fit within A4
        ticket_image = ticket_image.resize((int(pdf_width * 0.9), int(pdf_height * 0.45)), Image.LANCZOS)

        # Save ticket as temporary file and draw it on PDF
        ticket_path = f"{ticket_id}.png"
        ticket_image.save(ticket_path)
        c.drawImage(ticket_path, x=pdf_width * 0.05, y=pdf_height * 0.25, width=pdf_width * 0.9, height=pdf_height * 0.45)

        # Add a new page for the next ticket
        c.showPage()

        # Remove the temporary ticket image
        os.remove(ticket_path)

    c.save()
    print(f"{output_pdf} generated .")

# Example usage
prefix = "CONCERT2024"
start_number = 1
total_tickets = 500  # Adjust the number of tickets as needed
output_pdf = "concert_tickets.pdf"
#provided_image = "GosayeVip.JPG"  # Path to the provided image
provided_image = "provided_image.JPG"  # Path to the provided image

generate_tickets_pdf(prefix, start_number, total_tickets, output_pdf, provided_image)

concert_tickets.pdf generated successfully.
